In [2]:
from collections import Counter
from huggingface_hub import login
!pip install datasets
from datasets import load_dataset, Dataset
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
import numpy as np
from transformers import XLMRobertaModel, XLMRobertaTokenizerFast
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# authentication with huggingface
# create token at https://huggingface.co/settings/tokens (create as read)
# token should be stored locally, so technically login is only needed one time
login()

/home/wolfingten/.python_environments/ML/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/wolfingten/.python_environments/ML/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [30]:
# # download specific dataset files; entire archive is only ~250 mb
# articles = load_dataset("Cofacts/line-msg-fact-check-tw", "articles") # article contents and some meta info
# article_replies = load_dataset("Cofacts/line-msg-fact-check-tw", "article_replies") # 'join table' for articles and replies with added meta info

# open from csv
articles = pd.read_csv('/content/articles.csv',lineterminator='\n')
article_replies = pd.read_csv('/content/article_replies.csv')
art_rep_df = pd.merge(articles, article_replies, left_on="id", right_on="articleId", how="left")
art_rep_df.head()

,id,articleType,status_x,text,normalArticleReplyCount,createdAt_x,updatedAt_x,lastRequestedAt,userIdsha256_x,appId_x,...,articleId,replyId,userIdsha256_y,negativeFeedbackCount,positiveFeedbackCount,replyType,appId_y,status_y,createdAt_y,updatedAt_y
0,14h550ymn3m3u,TEXT,NORMAL,BA.5病毒引發的不只「縮短潛伏期，增加R0值」，還有易使人「再感染（Reinfecion）...,1.0,2022-09-05T05:26:33.360Z,2022-09-05T05:26:33.360Z,2022-09-05T05:26:33.381Z,3753ca22ca96f5fc1f13e62291f28e405d56b1a976b21d...,RUMORS_LINE_BOT,...,14h550ymn3m3u,6oo8DYMBv5it-Cx_8405,6892d0026181e95d034bf8781025afbf395d57cc88ac05...,0.0,3.0,RUMOR,WEBSITE,NORMAL,2022-09-05T10:40:41.789Z,2022-09-05T10:40:41.789Z
1,326xjpkjbf01i,TEXT,NORMAL,https://youtu.be/xK9NzL3PkdE,0.0,2022-09-08T12:34:27.602Z,2022-09-08T12:34:27.602Z,2022-09-08T12:34:27.634Z,243b5897c14f02fb5b92a9e4f4cc39d5fb84ff16173add...,RUMORS_LINE_BOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,oupi0eu9aocr,TEXT,NORMAL,被取消的航班\n\n2018年的一天，我從洛杉磯回國，乘坐凌晨一點的國航航班，馬上要登機了，...,1.0,2021-11-13T16:36:56.548Z,2021-11-13T16:36:56.548Z,2022-04-27T08:52:17.115Z,4a266a6fdefc88e59eef644402e97c7f4c8196becca686...,RUMORS_LINE_BOT,...,oupi0eu9aocr,MYotCYMBv5it-Cx_gIpH,fc9652aa205dab75e19ecff420945e95aa3909a54b0d9a...,0.0,1.0,OPINIONATED,WEBSITE,NORMAL,2022-09-04T15:45:20.464Z,2022-09-04T15:45:20.464Z
3,my5ep5z69tql,TEXT,NORMAL,你好👱🏻‍♀️\n\n簡單跟你說一下公司工作內容：\n我們是做購物系統訂單的工作，主要就是增...,1.0,2022-08-11T20:53:15.644Z,2022-08-11T20:53:15.644Z,2022-09-07T11:02:00.344Z,8a8a319bea0b3bff580d8507ae6470ba37e87791171886...,RUMORS_LINE_BOT,...,my5ep5z69tql,y22DlmsBFV14knB4ErTJ,d16417c7ce4ab67ac5a7901ce62621445db3e06da5da6f...,0.0,1.0,RUMOR,WEBSITE,NORMAL,2022-09-07T22:59:48.921Z,2022-09-07T22:59:48.921Z
4,3dnh713ikpf3c,TEXT,NORMAL,先和你簡單介紹\n我們誠徵線上打工小幫手\n工作內容有：聽寫打字、頁面排版、商品評論刷流量、...,1.0,2022-09-07T10:10:41.742Z,2022-09-07T10:10:41.742Z,2022-09-07T10:10:41.761Z,47e90b2538a5615b323f50571b77d91f7f105e55684aa1...,RUMORS_LINE_BOT,...,3dnh713ikpf3c,H8Hz4WsBqwaEkHKwtyFm,d16417c7ce4ab67ac5a7901ce62621445db3e06da5da6f...,0.0,1.0,RUMOR,WEBSITE,NORMAL,2022-09-07T23:00:19.207Z,2022-09-07T23:00:19.207Z


In [3]:
articles = load_dataset("Cofacts/line-msg-fact-check-tw", "articles") # article contents and some meta info
article_replies = load_dataset("Cofacts/line-msg-fact-check-tw", "article_replies") # 'join table' for articles and replies with added meta info
art_rep_df = pd.merge(articles["train"].to_pandas(), article_replies["train"].to_pandas(), left_on="id", right_on="articleId", how="left")
art_rep_df.head()

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

,id,articleType,status_x,text,normalArticleReplyCount,createdAt_x,updatedAt_x,lastRequestedAt,userIdsha256_x,appId_x,...,articleId,replyId,userIdsha256_y,negativeFeedbackCount,positiveFeedbackCount,replyType,appId_y,status_y,createdAt_y,updatedAt_y
0,14h550ymn3m3u,TEXT,NORMAL,BA.5病毒引發的不只「縮短潛伏期，增加R0值」，還有易使人「再感染（Reinfecion）...,1.0,2022-09-05T05:26:33.360Z,2022-09-05T05:26:33.360Z,2022-09-05T05:26:33.381Z,3753ca22ca96f5fc1f13e62291f28e405d56b1a976b21d...,RUMORS_LINE_BOT,...,14h550ymn3m3u,6oo8DYMBv5it-Cx_8405,6892d0026181e95d034bf8781025afbf395d57cc88ac05...,0.0,3.0,RUMOR,WEBSITE,NORMAL,2022-09-05T10:40:41.789Z,2022-09-05T10:40:41.789Z
1,326xjpkjbf01i,TEXT,NORMAL,https://youtu.be/xK9NzL3PkdE,0.0,2022-09-08T12:34:27.602Z,2022-09-08T12:34:27.602Z,2022-09-08T12:34:27.634Z,243b5897c14f02fb5b92a9e4f4cc39d5fb84ff16173add...,RUMORS_LINE_BOT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,oupi0eu9aocr,TEXT,NORMAL,被取消的航班\n\n2018年的一天，我從洛杉磯回國，乘坐凌晨一點的國航航班，馬上要登機了，...,1.0,2021-11-13T16:36:56.548Z,2021-11-13T16:36:56.548Z,2022-04-27T08:52:17.115Z,4a266a6fdefc88e59eef644402e97c7f4c8196becca686...,RUMORS_LINE_BOT,...,oupi0eu9aocr,MYotCYMBv5it-Cx_gIpH,fc9652aa205dab75e19ecff420945e95aa3909a54b0d9a...,0.0,1.0,OPINIONATED,WEBSITE,NORMAL,2022-09-04T15:45:20.464Z,2022-09-04T15:45:20.464Z
3,my5ep5z69tql,TEXT,NORMAL,你好👱🏻‍♀️\n\n簡單跟你說一下公司工作內容：\n我們是做購物系統訂單的工作，主要就是增...,1.0,2022-08-11T20:53:15.644Z,2022-08-11T20:53:15.644Z,2022-09-07T11:02:00.344Z,8a8a319bea0b3bff580d8507ae6470ba37e87791171886...,RUMORS_LINE_BOT,...,my5ep5z69tql,y22DlmsBFV14knB4ErTJ,d16417c7ce4ab67ac5a7901ce62621445db3e06da5da6f...,0.0,1.0,RUMOR,WEBSITE,NORMAL,2022-09-07T22:59:48.921Z,2022-09-07T22:59:48.921Z
4,3dnh713ikpf3c,TEXT,NORMAL,先和你簡單介紹\n我們誠徵線上打工小幫手\n工作內容有：聽寫打字、頁面排版、商品評論刷流量、...,1.0,2022-09-07T10:10:41.742Z,2022-09-07T10:10:41.742Z,2022-09-07T10:10:41.761Z,47e90b2538a5615b323f50571b77d91f7f105e55684aa1...,RUMORS_LINE_BOT,...,3dnh713ikpf3c,H8Hz4WsBqwaEkHKwtyFm,d16417c7ce4ab67ac5a7901ce62621445db3e06da5da6f...,0.0,1.0,RUMOR,WEBSITE,NORMAL,2022-09-07T23:00:19.207Z,2022-09-07T23:00:19.207Z


In [5]:
def remove_url(text: str):
    regex = r"https?\S+|\S*\.com\S*"
    text = re.sub("\n", " ", text) # also remove newlines. Beware trailing whitespaces
    text = re.sub(regex, "", text)
    if len(text) == 0:
        text = pd.NA
    return text

def filter_unicodes(texts: list):
    retain = list()
    remove = list()
    for t in texts:
        if pd.notna(t):
            regex = re.compile(r"[^\u4E00-\u9FFF\u3000-\u303F\u0000-\u007F\u2000-\u206F\uFF00-\uFF65]")
            filtered_txt = regex.sub("", t)
            retain.append(filtered_txt)
            deleted = regex.findall(t)
            remove.append(deleted)
        else:
            retain.append(t)
    remove = Counter([i for l in remove for i in l])
    return retain, remove

def find_unicodes(chars: list):
    out = {char : char.encode("unicode_escape") for char in chars}
    return out

# processing steps
clean_df = art_rep_df.copy()
clean_df = clean_df[clean_df["replyType"] != "NOT_ARTICLE"] # remove non articles
clean_df = clean_df.dropna(subset=["text", "replyType"]) # drop na's
clean_df["text"] = clean_df["text"].apply(remove_url) # delete urls
clean_df["text"], remove = filter_unicodes(clean_df["text"]) # filter for chinese and latin script
clean_df = clean_df.dropna(subset=["text", "replyType"])
clean_df.head()

In [6]:
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

listed = clean_df[['text','replyType']].dropna().copy()
listed['replyType'] = listed['replyType'].map(lambda x: x.replace("NOT_RUMOR", "0"))
listed['replyType'] = listed['replyType'].map(lambda x: x.replace("OPINIONATED", "1"))
listed['replyType'] = listed['replyType'].map(lambda x: x.replace("RUMOR", "2"))
listed['replyType'] = listed['replyType'].astype(int)
listed['tokenizer'] = [tokenizer(x[:500]) for x in listed['text']]
listed['tokens'] = [torch.tensor(x.input_ids, dtype=torch.long) for x in listed['tokenizer']]
listed['attention'] = [torch.tensor(x.attention_mask, dtype=torch.long) for x in listed['tokenizer']]
listed['replyType'] = listed['replyType'].map(lambda x: torch.tensor(x, dtype=torch.long))
#padded = torch.nn.utils.rnn.pad_sequence(listed['text'], batch_first=True)
padded_tokens = torch.nn.utils.rnn.pad_sequence([i.flip(dims=[0]) for i in listed['tokens']], batch_first=True).flip(dims=[0])
padded_attention_mask = torch.nn.utils.rnn.pad_sequence([i.flip(dims=[0]) for i in listed['attention']], batch_first=True).flip(dims=[0])
zipped = list(zip(listed['tokens'], listed['replyType'], listed['attention']))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [34]:
# def k_fold_split_data(dataset, batch_size, k=5):
#     n = len(dataset)
#     fold_size = n // k
#     folds = []

#     def collate_fn(data):
#         tensors, targets = zip(*data)
#         features = torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)
#         targets = torch.stack(targets)
#         return features, targets

#     for i in range(k):
#         start = i * fold_size
#         end = (i + 1) * fold_size if i < k - 1 else n
#         folds.append(torch.utils.data.Subset(dataset, range(start, end)))

#     dataloaders = []
#     for i in range(k):
#         validation_dataset = folds[i]
#         train_folds = [folds[j] for j in range(k) if j != i]
#         train_dataset = torch.utils.data.ConcatDataset(train_folds)

#         y = torch.tensor([label for _, label in train_dataset], dtype=torch.long)

#         global class_weights
#         class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y.numpy())
#         class_weights = torch.tensor(class_weights, dtype=torch.float)

#         train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
#         validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
#         dataloaders.append((train_dataloader, validation_dataloader))

#     return dataloaders

In [7]:
def randomly_split_data(dataset, batch_size):


    #generator = torch.Generator().manual_seed(42)
    #train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1], generator=generator)
    train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1])

    y = torch.tensor([label for _, label, _ in dataset], dtype=torch.long)

    def collate_fn(data):
        tensors, targets, attention_mask = zip(*data)
        features = torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)
        targets = torch.stack(targets)
        attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True)
        return features, targets, attention_mask

    global class_weights
    class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y.numpy())
    class_weights = torch.tensor(class_weights, dtype=torch.float)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    return train_dataloader, validation_dataloader, test_dataloader

In [8]:
class TuneableModel(torch.nn.Module):
    def __init__(self, input_size, layer_size, dropout_rate, n_layers):
        super(TuneableModel, self).__init__()
        self.roberta = XLMRobertaModel.from_pretrained("xlm-roberta-base")
        self.requires_grad_(False)
        self.lstm = torch.nn.LSTM(input_size=32, hidden_size=layer_size, bidirectional=False,
                                  num_layers=n_layers, batch_first=True, dropout=dropout_rate, proj_size=3)
        self.downshift = torch.nn.Linear(768, 32)
        # self.output_layer = torch.nn.Linear(1, 3)
        #self.batchnorm = torch.nn.BatchNorm1d(32)
        self.activation = torch.nn.ReLU()
        # self.linear = torch.nn.Linear(layer_size, layer_size)

    def forward(self, x, attention_mask):
        x = self.roberta(x, attention_mask=attention_mask).last_hidden_state
        x = self.downshift(x)
        #x = self.batchnorm(x)
        x = self.activation(x)
        x = self.lstm(x)
        #print(x[0].shape)
        #x = self.activation(x)
        # x = self.linear(x)
        # x = self.batchnorm(x)
        # x = self.activation(x)
        # x = self.output_layer(x[0][:,-1:])
        return x[0][:,-1,:]

In [9]:
def train_test(model, dataloader, optimizer, training="train"):

    loss_function = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))

    if training == "train":
        model.train()
    elif training == "validation":
        model.eval()
    elif training == "test":
        model.eval()
    else:
        raise ValueError("training argument must be either 'train', 'validation' or 'test'")

    cumulative_loss = 0
    prediction_list = []
    label_list = []
    for sample in tqdm(dataloader):
        if training == "train":
            optimizer.zero_grad()
        input, targets, attention_mask = sample[0].to(device).long(), sample[1].to(device), sample[2].to(device)
        output = model(input, attention_mask).to(device)
        loss_value = loss_function(output, targets)
        cumulative_loss += loss_value.item()
        if training == "train":
            print("batch loss:", loss_value.sum().item())
            loss_value.sum().backward()
            optimizer.step()

        predictions = output.to('cpu').detach().numpy().argmax(axis=1)
        target_labels = sample[1]
        prediction_list.extend(predictions)
        label_list.extend(target_labels)
    #f1 = f1_score(label_list, prediction_list)
    accuracy = accuracy_score(label_list, prediction_list)
    #confusion = confusion_matrix(label_list, prediction_list)

    return cumulative_loss, accuracy#, f1, confusion

In [10]:
# Training sample
def evaluate(params, dataset):
    dropout, hidden_size, learning_rate, batch_size, n_hidden = params

    max_epochs = 1000
    max_patience = 5
    seed = 42

    accuracies = []
    f1s = []
    train_dataloader, validation_dataloader, test_dataloader = randomly_split_data(dataset, batch_size)
    #dataloaders = k_fold_split_data(dataset, batch_size, k=5)
    # train_dataloader, validation_dataloader = dataloader[0], dataloader[1]
    # test_dataloader = dataloader[1]
    PATH = "model_.pt"
    last_loss = 1000000
    torch.manual_seed(seed)
    input_size = train_dataloader.dataset[0][0].size()[0]
    model = TuneableModel(input_size, hidden_size, dropout, n_hidden)
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    for epoch in range(max_epochs):
        # training
        train_loss, train_accuracy = train_test(model, train_dataloader, optimizer, training="train")
        train_loss, train_accuracy = train_loss, round(train_accuracy, 4)
        # validation at end of epoch
        validation_loss, validation_accuracy = train_test(model, validation_dataloader, optimizer, training="validation")
        validation_loss, validation_accuracy = validation_loss, round(validation_accuracy, 4)
        if validation_loss < last_loss:
            last_loss = validation_loss
            current_patience = 0
        else:
            if current_patience == 0:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': last_loss,
                    }, PATH)
            current_patience += 1
        if current_patience == max_patience:
            break
        if epoch % 1 == 0:
            print(f"Epoch {epoch} validation loss: {validation_loss} validation accuracy: {validation_accuracy*100}%")
    # Testing once patience is reached
    torch.manual_seed(seed)
    model = TuneableModel(input_size, hidden_size, dropout, n_hidden)
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    test_loss, test_accuracy = train_test(model, test_dataloader, optimizer, training="test")
    test_loss, test_accuracy = test_loss, test_accuracy
    #print(f"Model {i} at epoch {checkpoint['epoch']} test results: accuracy: {test_accuracy*100}% f1: {test_f1}")
    # accuracies.append(test_accuracy)
    # f1s.append(test_f1)
    #print(test_confusion)

    return round(test_accuracy*100, 2)
    # print(f"Average accuracy: {round(np.mean(accuracies), 2)}%")
    # print(f"Average f1: {round(np.mean(f1s), 2)}")


In [11]:
dataset = zipped
params = (0.0, 10, 0.01, 32, 1)
accuracy= evaluate(params, dataset)
print(f"final test accuracy: {accuracy}%")

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  0%|          | 1/1657 [00:02<1:06:11,  2.40s/it]

batch loss: 1.1014227867126465


  0%|          | 2/1657 [00:04<1:00:02,  2.18s/it]

batch loss: 1.11203932762146


...

batch loss: 1.0872187614440918


  7%|▋         | 121/1657 [04:08<54:23,  2.12s/it]

batch loss: 1.0957530736923218


  7%|▋         | 121/1657 [04:10<52:57,  2.07s/it]


KeyboardInterrupt: 